### 1. data loader 

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np


from keras.preprocessing.image import img_to_array
class loader:
    def __init__(self):
        self.x_data=[]
        self.y_data=[]

    def load(self,path):
        files=os.listdir(path)
        for i in files:
            img=cv2.imread(path+'/'+i,cv2.IMREAD_GRAYSCALE)# 400 x 600
            img=cv2.resize(img,(240,240))
            #img=cv2.resize(img,(240,240))
            #print(img.shape)
            self.y_data.append(img_to_array(img))
            self.x_data.append(img_to_array(img))
        return self.x_data,self.y_data



img=cv2.imread('../input/grid-map/111202511.png',cv2.IMREAD_GRAYSCALE)# 400 x 600
img=cv2.resize(img,(240,240)) # 宽 x 高
plt.imshow(img, cmap=plt.cm.gray)
plt.xticks([])  # 去掉x轴
plt.yticks([])  # 去掉y轴
plt.axis('off')  # 去掉坐标轴
plt.show()

In [2]:
from keras.models import Model
from keras.layers import Conv2D,UpSampling2D,MaxPooling2D
from keras.layers import Input
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras import regularizers

class Models:
    def __init__(self,w,h,c):
        self.w=w
        self.h=h
        self.c=c
    def Arch1(self):
        inp=Input(shape=(self.w,self.h,self.c))
        enc=Conv2D(64,(3,3),padding='same')(inp)
        enc=BatchNormalization()(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(32,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(16,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(8,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)


        dec=Conv2D(8,(3,3),padding='same')(enc)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(16,(3,3),padding='same')(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(32,(3,3),padding='same')(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(64,(3,3),padding='same')(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        final=Conv2D(1,(3,3),padding='same',activation='sigmoid')(dec)
        auto=Model(inp,final)
        return auto

    def Arch2(self):
        inp=Input(shape=(self.w,self.h,self.c))
        enc=Conv2D(64,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(inp)
        enc=BatchNormalization()(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(32,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(16,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2),activity_regularizer=regularizers.l1(10e-5))(enc)
        enc=Conv2D(8,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)


        dec=Conv2D(8,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(enc)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(16,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(32,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(64,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        final=Conv2D(1,(3,3),padding='same',activation='sigmoid')(dec)
        auto=Model(inp,final)
        return auto


## main 函数：

In [3]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam_v2

w=240
h=240
c=1

## 数据载入、训练集和测试集划分
load_img=loader()
x_data,y_data=load_img.load('../input/grid-map')
x_data=np.array(x_data,dtype='float')/255.0
y_data=np.array(y_data,dtype='float')/255.0
train_x,test_x,train_y,test_y=train_test_split(x_data,y_data,test_size=0.1,random_state=30)

In [4]:
## 模型载入
model=Models(w,h,c)
auto_encoder=model.Arch2()
auto_encoder.summary()

auto_encoder.compile(optimizer='adadelta',loss='binary_crossentropy')
auto_encoder.fit(train_x,train_y,batch_size=3,shuffle='true',epochs=200,validation_data=(test_x,test_y),verbose=1)
auto_encoder.save('noise.MODEL')

opt=adam_v2.Adam(learning_rate=0.01,decay=0.01/50)


2022-05-06 03:33:34.641426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 03:33:34.755604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 03:33:34.757069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 240, 240, 64)      640       
_________________________________________________________________
batch_normalization (BatchNo (None, 240, 240, 64)      256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 240, 240, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 120, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 120, 32)      18464     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 120, 120, 32)      0     

2022-05-06 03:33:34.760157: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 03:33:34.763777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 03:33:34.765128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 03:33:34.766434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/200


2022-05-06 03:33:41.849646: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


572/572 [==============================] - 19s 18ms/step - loss: 80.1458 - val_loss: 64.2032
Epoch 2/200
572/572 [==============================] - 9s 16ms/step - loss: 64.2607 - val_loss: 57.5271
Epoch 3/200
572/572 [==============================] - 10s 17ms/step - loss: 52.9108 - val_loss: 48.7131
Epoch 4/200
572/572 [==============================] - 10s 17ms/step - loss: 45.7351 - val_loss: 42.8631
Epoch 5/200
572/572 [==============================] - 10s 17ms/step - loss: 40.8579 - val_loss: 38.8705
Epoch 6/200
572/572 [==============================] - 10s 17ms/step - loss: 37.1587 - val_loss: 35.5414
Epoch 7/200
572/572 [==============================] - 10s 18ms/step - loss: 33.9854 - val_loss: 32.5246
Epoch 8/200
572/572 [==============================] - 10s 18ms/step - loss: 31.1043 - val_loss: 29.7867
Epoch 9/200
572/572 [==============================] - 10s 17ms/step - loss: 28.4832 - val_loss: 27.3387
Epoch 10/200
572/572 [==============================] - 10s 18ms/ste

2022-05-06 04:06:31.232026: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
